In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, models, callbacks
import tensorflow_hub as hub
import pandas as pd

# 載入資料

In [ ]:
# 將訓練資料拆分成7:3，分別是訓練與測試集，並且取得Info
[train_ds, valid_ds, test_ds], info = tfds.load('colorectal_histology',
                            split=['train[:70%]','train[70%:85%]', 'train[85%:]'],
                            shuffle_files=True,
                            with_info=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/5000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/colorectal_histology/2.0.0.incomplete5RKJRL/colorectal_histology-train.tfr…

Dataset colorectal_histology downloaded and prepared to /root/tensorflow_datasets/colorectal_histology/2.0.0. Subsequent calls will reuse this data.


In [ ]:
# 參數設定
#input_shapes = info.features[info.supervised_keys[0]].shape
input_shapes = (224,224,3)
img_size = input_shapes[:2]
batch_size = 32
epoch = 50

In [ ]:
# 預處理函數
def preprocess_data(data):
    image = data['image']
    label = data['label']
    # 將圖片大小調整為指定大小,並正規化像素值至 [0, 1]
    image = tf.image.resize(image, img_size) / 255.0
    return image, label

# 將預處理函數應用到數據集，並將資料分批
train_data = train_ds.map(preprocess_data).batch(batch_size)
valid_data = valid_ds.map(preprocess_data).batch(batch_size)
test_data = test_ds.map(preprocess_data).batch(batch_size)

# 定義模型

In [ ]:
# 定義 MLP 模型
model_1 = models.Sequential()
model_1.add(layers.Flatten(input_shape=input_shapes))
model_1.add(layers.Dense(128, activation='relu'))
model_1.add(layers.Dense(64, activation='relu'))
model_1.add(layers.Dense(info.features['label'].num_classes, activation='softmax'))
model_1.summary()
# 編譯模型
model_1.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 150528)            0         
                                                                 
 dense (Dense)               (None, 128)               19267712  
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 8)                 520       
                                                                 
Total params: 19276488 (73.53 MB)
Trainable params: 19276488 (73.53 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# 定義 CNN 模型
model_2 = models.Sequential()
model_2.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shapes))
model_2.add(layers.MaxPooling2D((2, 2)))
model_2.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_2.add(layers.MaxPooling2D((2, 2)))
model_2.add(layers.Conv2D(128, (3, 3), activation='relu'))
model_2.add(layers.MaxPooling2D((2, 2)))
model_2.add(layers.Flatten())
model_2.add(layers.Dense(128, activation='relu'))
model_2.add(layers.Dense(info.features['label'].num_classes, activation='softmax'))
model_2.summary()
# 編譯模型
model_2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 26, 26, 128)      

In [ ]:
# 定義 CNN 模型 (用於callbacks)
model_3 = models.Sequential()
model_3.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shapes))
model_3.add(layers.MaxPooling2D((2, 2)))
model_3.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_3.add(layers.MaxPooling2D((2, 2)))
model_3.add(layers.Conv2D(128, (3, 3), activation='relu'))
model_3.add(layers.MaxPooling2D((2, 2)))
model_3.add(layers.Flatten())
model_3.add(layers.Dense(128, activation='relu'))
model_3.add(layers.Dense(info.features['label'].num_classes, activation='softmax'))
model_3.summary()
# 編譯模型
model_3.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 111, 111, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 26, 26, 128)      

In [ ]:
# 使用 TensorFlow Hub 中的 MobileNetV2 預訓練模型
MNV2_model = hub.load("https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4")
# 凍結預訓練模型的權重
MNV2_model.trainable = False

model_4 = models.Sequential()
model_4.add(hub.KerasLayer(MNV2_model, trainable=False, input_shape=input_shapes))
model_4.add(layers.Flatten())
model_4.add(layers.Dense(128, activation='relu'))
model_4.add(layers.Dense(info.features['label'].num_classes, activation='softmax'))
model_4.summary()
# 編譯模型
model_4.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1001)              3540265   
                                                                 
 flatten_3 (Flatten)         (None, 1001)              0         
                                                                 
 dense_7 (Dense)             (None, 128)               128256    
                                                                 
 dense_8 (Dense)             (None, 8)                 1032      
                                                                 
Total params: 3669553 (14.00 MB)
Trainable params: 129288 (505.03 KB)
Non-trainable params: 3540265 (13.51 MB)
_________________________________________________________________


In [ ]:
# 建立 VGG19 模型
VGG19_model = tf.keras.applications.VGG19(input_shape=input_shapes, include_top=False, weights='imagenet')
# 凍結 VGG19 的權重
VGG19_model.trainable = False

model_5 = models.Sequential(VGG19_model)
model_5.add(layers.Flatten())
model_5.add(layers.Dense(256, activation='relu'))
model_5.add(layers.Dropout(0.5))
model_5.add(layers.Dense(info.features['label'].num_classes, activation='softmax'))
model_5.summary()
# 編譯模型
model_5.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

80134624/80134624 [==============================] - 4s 0us/step
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 7, 7, 512)         20024384  
                                                                 
 flatten_4 (Flatten)         (None, 25088)             0         
                                                                 
 dense_9 (Dense)             (None, 256)               6422784   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_10 (Dense)            (None, 8)                 2056      
                                                                 
Total params: 26449224 (100.90 MB)
Trainable params: 6424840 (24.51 MB)
Non-trainable params: 20024384 (76.39 MB)
_______

In [ ]:
# 建立 DenseNet121 模型
DN121_model = tf.keras.applications.DenseNet121(input_shape=input_shapes, include_top=False, weights='imagenet')
# 凍結 DenseNet121 的權重
DN121_model.trainable = False

model_6 = models.Sequential(DN121_model)
model_6.add(layers.GlobalAveragePooling2D())
model_6.add(layers.Dense(256, activation='relu'))
model_6.add(layers.Dropout(0.5))
model_6.add(layers.Dense(info.features['label'].num_classes, activation='softmax'))
model_6.summary()
# 編譯模型
model_6.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

29084464/29084464 [==============================] - 2s 0us/step
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet121 (Functional)    (None, 7, 7, 1024)        7037504   
                                                                 
 global_average_pooling2d (  (None, 1024)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense_11 (Dense)            (None, 256)               262400    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_12 (Dense)            (None, 8)                 2056      
                                                                 
Total params: 7301960 (27.85 MB)
Trainable params: 2644

# 訓練模型

In [ ]:
# MLP model
history_1 = model_1.fit(train_data,
                        epochs=epoch,
                        validation_data=valid_data)

Epoch 1/50
110/110 [==============================] - 8s 54ms/step - loss: 18.8299 - accuracy: 0.2226 - val_loss: 9.6424 - val_accuracy: 0.2547
Epoch 2/50
110/110 [==============================] - 7s 64ms/step - loss: 7.3614 - accuracy: 0.3071 - val_loss: 4.3040 - val_accuracy: 0.3627
Epoch 3/50
110/110 [==============================] - 6s 56ms/step - loss: 3.1473 - accuracy: 0.3709 - val_loss: 4.0324 - val_accuracy: 0.2893
Epoch 4/50
110/110 [==============================] - 5s 44ms/step - loss: 2.8857 - accuracy: 0.4114 - val_loss: 3.4833 - val_accuracy: 0.3013
Epoch 5/50
110/110 [==============================] - 5s 46ms/step - loss: 2.9344 - accuracy: 0.4169 - val_loss: 2.8687 - val_accuracy: 0.3573
Epoch 6/50
110/110 [==============================] - 5s 46ms/step - loss: 2.2311 - accuracy: 0.4711 - val_loss: 1.7703 - val_accuracy: 0.5027
Epoch 7/50
110/110 [==============================] - 6s 57ms/step - loss: 1.9098 - accuracy: 0.4834 - val_loss: 1.4992 - val_accuracy: 0.442

In [ ]:
# CNN model
history_2 = model_2.fit(train_data,
                        epochs=epoch,
                        validation_data=valid_data)

Epoch 1/50
110/110 [==============================] - 13s 69ms/step - loss: 1.3180 - accuracy: 0.4623 - val_loss: 0.9219 - val_accuracy: 0.6227
Epoch 2/50
110/110 [==============================] - 6s 55ms/step - loss: 0.7744 - accuracy: 0.6794 - val_loss: 0.7826 - val_accuracy: 0.7267
Epoch 3/50
110/110 [==============================] - 7s 67ms/step - loss: 0.6567 - accuracy: 0.7423 - val_loss: 0.8418 - val_accuracy: 0.6867
Epoch 4/50
110/110 [==============================] - 7s 66ms/step - loss: 0.5349 - accuracy: 0.7917 - val_loss: 0.8539 - val_accuracy: 0.6720
Epoch 5/50
110/110 [==============================] - 6s 57ms/step - loss: 0.4440 - accuracy: 0.8306 - val_loss: 0.8564 - val_accuracy: 0.6613
Epoch 6/50
110/110 [==============================] - 7s 65ms/step - loss: 0.4041 - accuracy: 0.8463 - val_loss: 0.7933 - val_accuracy: 0.7253
Epoch 7/50
110/110 [==============================] - 6s 56ms/step - loss: 0.3146 - accuracy: 0.8849 - val_loss: 1.2678 - val_accuracy: 0.660

In [ ]:
# 有使用callback的CNN

# EarlyStopping設定
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
history_3 = model_3.fit(train_data,
                        epochs=epoch,
                        validation_data=valid_data,
                        callbacks=[early_stopping])

Epoch 1/50
110/110 [==============================] - 9s 70ms/step - loss: 1.3694 - accuracy: 0.4537 - val_loss: 0.9270 - val_accuracy: 0.5893
Epoch 2/50
110/110 [==============================] - 6s 57ms/step - loss: 0.8460 - accuracy: 0.6389 - val_loss: 0.8306 - val_accuracy: 0.6613
Epoch 3/50
110/110 [==============================] - 8s 69ms/step - loss: 0.7131 - accuracy: 0.7057 - val_loss: 0.7947 - val_accuracy: 0.6440
Epoch 4/50
110/110 [==============================] - 7s 66ms/step - loss: 0.5838 - accuracy: 0.7711 - val_loss: 0.6761 - val_accuracy: 0.7293
Epoch 5/50
110/110 [==============================] - 6s 56ms/step - loss: 0.5339 - accuracy: 0.7886 - val_loss: 0.7100 - val_accuracy: 0.7067
Epoch 6/50
110/110 [==============================] - 7s 68ms/step - loss: 0.4780 - accuracy: 0.8154 - val_loss: 0.8256 - val_accuracy: 0.6387
Epoch 7/50
110/110 [==============================] - 6s 57ms/step - loss: 0.4240 - accuracy: 0.8351 - val_loss: 0.8582 - val_accuracy: 0.7373

In [ ]:
# 使用TFhub中 MobileNetV2 model
history_4 = model_4.fit(train_data,
                        epochs=epoch,
                        validation_data=valid_data)

Epoch 1/50
110/110 [==============================] - 18s 81ms/step - loss: 0.5489 - accuracy: 0.8094 - val_loss: 0.3970 - val_accuracy: 0.8747
Epoch 2/50
110/110 [==============================] - 6s 55ms/step - loss: 0.2730 - accuracy: 0.9083 - val_loss: 0.3499 - val_accuracy: 0.8840
Epoch 3/50
110/110 [==============================] - 8s 69ms/step - loss: 0.1913 - accuracy: 0.9363 - val_loss: 0.3595 - val_accuracy: 0.8813
Epoch 4/50
110/110 [==============================] - 6s 56ms/step - loss: 0.1360 - accuracy: 0.9583 - val_loss: 0.3674 - val_accuracy: 0.8867
Epoch 5/50
110/110 [==============================] - 6s 54ms/step - loss: 0.1028 - accuracy: 0.9700 - val_loss: 0.3696 - val_accuracy: 0.8893
Epoch 6/50
110/110 [==============================] - 6s 55ms/step - loss: 0.0752 - accuracy: 0.9794 - val_loss: 0.3985 - val_accuracy: 0.8893
Epoch 7/50
110/110 [==============================] - 7s 63ms/step - loss: 0.0600 - accuracy: 0.9829 - val_loss: 0.4437 - val_accuracy: 0.874

In [ ]:
# 使用 VGG19
history_5 = model_5.fit(train_data,
                        epochs=epoch,
                        validation_data=valid_data)

Epoch 1/50
110/110 [==============================] - 39s 271ms/step - loss: 1.3758 - accuracy: 0.5451 - val_loss: 0.7428 - val_accuracy: 0.7907
Epoch 2/50
110/110 [==============================] - 20s 178ms/step - loss: 0.8801 - accuracy: 0.6709 - val_loss: 0.6248 - val_accuracy: 0.8240
Epoch 3/50
110/110 [==============================] - 20s 179ms/step - loss: 0.7435 - accuracy: 0.7171 - val_loss: 0.5402 - val_accuracy: 0.8333
Epoch 4/50
110/110 [==============================] - 20s 181ms/step - loss: 0.6557 - accuracy: 0.7489 - val_loss: 0.5121 - val_accuracy: 0.8520
Epoch 5/50
110/110 [==============================] - 20s 182ms/step - loss: 0.6341 - accuracy: 0.7586 - val_loss: 0.5248 - val_accuracy: 0.8400
Epoch 6/50
110/110 [==============================] - 22s 198ms/step - loss: 0.6059 - accuracy: 0.7623 - val_loss: 0.4481 - val_accuracy: 0.8547
Epoch 7/50
110/110 [==============================] - 20s 183ms/step - loss: 0.5596 - accuracy: 0.7849 - val_loss: 0.4447 - val_ac

In [ ]:
# 使用 DenseNet121
history_6 = model_6.fit(train_data,
                        epochs=epoch,
                        validation_data=valid_data)

Epoch 1/50
110/110 [==============================] - 31s 188ms/step - loss: 0.7873 - accuracy: 0.7194 - val_loss: 0.4146 - val_accuracy: 0.8640
Epoch 2/50
110/110 [==============================] - 12s 109ms/step - loss: 0.4332 - accuracy: 0.8534 - val_loss: 0.3408 - val_accuracy: 0.8867
Epoch 3/50
110/110 [==============================] - 13s 114ms/step - loss: 0.3451 - accuracy: 0.8774 - val_loss: 0.3161 - val_accuracy: 0.8960
Epoch 4/50
110/110 [==============================] - 13s 114ms/step - loss: 0.3147 - accuracy: 0.8914 - val_loss: 0.3053 - val_accuracy: 0.9013
Epoch 5/50
110/110 [==============================] - 13s 115ms/step - loss: 0.2730 - accuracy: 0.9037 - val_loss: 0.2924 - val_accuracy: 0.9080
Epoch 6/50
110/110 [==============================] - 13s 121ms/step - loss: 0.2448 - accuracy: 0.9174 - val_loss: 0.2926 - val_accuracy: 0.9027
Epoch 7/50
110/110 [==============================] - 12s 109ms/step - loss: 0.2237 - accuracy: 0.9220 - val_loss: 0.2862 - val_ac

# 評估模型

In [ ]:
model_list = [model_1, model_2, model_3, model_4, model_5, model_6]
result = pd.DataFrame(columns=['Accuracy', 'Loss'])

for i, model in enumerate(model_list):
  test_loss, test_acc = model.evaluate(test_data)
  result = result.append({'Accuracy': test_acc, 'Loss': test_loss}, ignore_index=True)

 7/24 [=======>......................] - ETA: 0s - loss: 1.7085 - accuracy: 0.6205

<ipython-input-17-72cb0431e5e8>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append({'Accuracy': test_acc, 'Loss': test_loss}, ignore_index=True)


 7/24 [=======>......................] - ETA: 0s - loss: 0.6247 - accuracy: 0.7768

<ipython-input-17-72cb0431e5e8>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append({'Accuracy': test_acc, 'Loss': test_loss}, ignore_index=True)


 3/24 [==>...........................] - ETA: 0s - loss: 0.1801 - accuracy: 0.9479

...

<ipython-input-17-72cb0431e5e8>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append({'Accuracy': test_acc, 'Loss': test_loss}, ignore_index=True)


 1/24 [>.............................] - ETA: 2s - loss: 0.0724 - accuracy: 0.9688

<ipython-input-17-72cb0431e5e8>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append({'Accuracy': test_acc, 'Loss': test_loss}, ignore_index=True)


24/24 [==============================] - 2s 85ms/step - loss: 0.2055 - accuracy: 0.9440


<ipython-input-17-72cb0431e5e8>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append({'Accuracy': test_acc, 'Loss': test_loss}, ignore_index=True)


In [ ]:
result.index = ['MLP', 'CNN(無callbacks)', 'CNN(有callbacks)', 'MobileNetV2', 'VGG19', 'DenseNet121']

result

,Accuracy,Loss
MLP,0.472000,2.040691
CNN(無callbacks),0.600000,1.854572
CNN(有callbacks),0.758667,0.616946
MobileNetV2,0.913333,0.406884
VGG19,0.834667,0.646118
DenseNet121,0.944000,0.205501
